In [72]:
import pandas as pd
import ast
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer
df = pd.read_csv("../Database/Dataset.csv")
df_raw = df.copy()  # Keep a backup of the original dataset

In [73]:
df.shape

(1266348, 24)

In [74]:
print(df.columns.tolist())

['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date', 'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'tagline', 'genres', 'production_companies', 'production_countries', 'spoken_languages', 'keywords']


In [75]:
# Initial Data Cleaning - Remove Columns with Too Many Null Values

# 1. Threshold setzen ( hier 50 % )
# Falls eine Spalte mehr als 50 % Null Values hat, wird sie deleted.
# Es bleiben nur Cols, die weniger als 50 % Null Values haben.
threshold = len(df) * 0.5

# Zähle die Null values pro Spalte
null_counts = df.isnull().sum()


# 2. Spalten ermitteln, bei denen null_counts > threshold
cols_to_drop = null_counts[null_counts > threshold].index.tolist()

# 3. Spalten löschen
df = df.drop(columns=cols_to_drop)

print("Original shape:", df_raw.shape)
print("Cleaned shape:", df.shape)
print("----------------------------------------\n")

# To Checken was gedropped wurde:
# dropped_cols als Set Difference
dropped_cols = set(df_raw.columns) - set(df.columns)
print("Dropped columns:", dropped_cols)
print("----------------------------------------")
print("----------------------------------------")
print("----------------------------------------")
print("Remaining columns:", df.columns.tolist())

Original shape: (1266348, 24)
Cleaned shape: (1266348, 19)
----------------------------------------

Dropped columns: {'production_companies', 'backdrop_path', 'keywords', 'homepage', 'tagline'}
----------------------------------------
----------------------------------------
----------------------------------------
Remaining columns: ['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date', 'revenue', 'runtime', 'adult', 'budget', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'genres', 'production_countries', 'spoken_languages']


In [76]:
# Second Data Cleaning
# Wir Droppen die Spalten, die wir nicht brauchen

cols_to_drop_for_model = [
    'id',
    'title',
    'original_title',
    'imdb_id',
    'poster_path',
    'overview',
    'vote_average',
    'production_countries', # already using original language and genres
    'spoken_languages' # already using original language
]

features = df.drop(columns=cols_to_drop_for_model)
target = df['vote_average']

print("Original DataFrame:", df.shape)
print("Features DataFrame:", features.shape)
print("-------------------------------------------")
print("The Features we are going to be using:", features.columns.tolist())



Original DataFrame: (1266348, 19)
Features DataFrame: (1266348, 10)
-------------------------------------------
The Features we are going to be using: ['vote_count', 'status', 'release_date', 'revenue', 'runtime', 'adult', 'budget', 'original_language', 'popularity', 'genres']


In [77]:
# check values in different columns

#print(df['status'].value_counts(dropna=False))
#print(df['adult'].value_counts(dropna=False))
#print(df['original_language'].value_counts())
df['genres'].describe()

count          733267
unique          14126
top       Documentary
freq           147108
Name: genres, dtype: object

In [78]:
# https://stackoverflow.com/questions/58033652/separating-categories-within-one-column-in-my-dataframe?utm_source=chatgpt.com

def extract_genres(genre_str):
    try:
        genres = ast.literal_eval(genre_str)
        return [g['name'] for g in genres]
    except:
        return []

df['genre_list'] = df['genres'].apply(extract_genres)



mlb = MultiLabelBinarizer()
df_genres = pd.DataFrame(
    mlb.fit_transform(df['genre_list']),
    columns=[f"has_{g}" for g in mlb.classes_],
    index=df.index
)
df = pd.concat([df, df_genres], axis=1)

In [79]:
# Feature Engineering
scaler = MinMaxScaler()

# Convert vote_count using logarithmic scaling
# prevents extreme large values
df['vote_count_log'] = np.log1p(df['vote_count'])

# Use One Hot Encoding for the STATUS
df['status_released'] = (df['status'] == 'Released').astype(int)
# By Far the majority of movies are Released -> Binary Choice
# 1 : Released, 0 : Not Released


# Convert 'release_date' to datetime and extract the YEAR
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
# coerce -> If parsing fails, set the value to NaT (Not a Time)
df['release_year'] = df['release_date'].dt.year
df['release_year'] = df['release_year'].clip(upper=2025).clip(lower=1900)

# Normalize BUDGET and REVENUE
# Fill NaN values with 0 for budget and revenue
df['budget'] = df['budget'].fillna(0)
df['revenue'] = df['revenue'].fillna(0)

# Scale budget and revenue between 0 and 1
df[['budget_scaled', 'revenue_scaled']] = scaler.fit_transform(df[['budget', 'revenue']])


# Convert RUNTIME to numeric, fill NaN with median
df['runtime'] = df['runtime'].fillna(df['runtime'].median())
df[['runtime_scaled']] = scaler.fit_transform(df[['runtime']])


# Convert ADULT to binary (1 for True, 0 for False)
# actually is statistically releveant, as almost 10 % are adult movies
df['adult'] = df['adult'].astype(bool).astype(int)


# One Hot Encoding for ORIGINAL_LANGUAGE
df = pd.get_dummies(df, columns=['original_language'], prefix='lang', drop_first=True)


# Min Max Scaling for POPULARITY
df[['popularity_scaled']] = scaler.fit_transform(df[['popularity']])


# Extracting features for genres with MultiLabelBinarizer in function above
genre_features = [col for col in df.columns if col.startswith('has_')]

# Final list of features
feature_cols = [
    'vote_count_log',
    'status_released',
    'release_year',
    'budget_scaled',
    'revenue_scaled',
    'runtime_scaled',
    'adult',
    'popularity_scaled',
    #'spoken_languages' — handled separately
] + genre_features + [col for col in df.columns if col.startswith('lang_')]


features = df[feature_cols]

In [80]:
features.describe()

,vote_count_log,status_released,release_year,budget_scaled,revenue_scaled,runtime_scaled,adult,popularity_scaled
count,1.266348e+06,1.266348e+06,1.022224e+06,1.266348e+06,1.266348e+06,1.266348e+06,1.266348e+06,1.266348e+06
mean,4.856530e-01,9.711951e-01,2.000457e+03,2.541705e-04,1.317004e-04,5.122499e-03,9.818470e-02,3.778678e-04
std,1.070297e+00,1.672578e-01,2.651279e+01,5.188038e-03,3.764931e-03,4.246285e-03,2.975643e-01,2.403095e-03
min,0.000000e+00,0.000000e+00,1.900000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,1.000000e+00,1.990000e+03,0.000000e+00,2.400000e-09,1.940671e-03,0.000000e+00,2.003769e-04
50%,0.000000e+00,1.000000e+00,2.011000e+03,0.000000e+00,2.400000e-09,3.257555e-03,0.000000e+00,2.003769e-04
75%,6.931472e-01,1.000000e+00,2.019000e+03,0.000000e+00,2.400000e-09,7.970613e-03,0.000000e+00,2.805277e-04
max,1.044860e+01,1.000000e+00,2.025000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
